<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Bloxorz_full_button.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import copy
import time
import os
import random
import numpy as np

In [ ]:
# Create a dictionary to save memory later
move = {None: "Initial Block", 0 : "NONE", -1 : "UP", 1 : "DOWN", -2 : "LEFT", 2 : "RIGHT"}
state = {0 : "STANDING", 1 : "LYING_HORIZONTALLY", 2 : "LYING_VERTICALLY", 3 : "SPLIT"}
square = {0 : "NONE", 1 : "WEAK", 2 : "STRONG", 3 : "WEAK_SWITCH", 4 : "STRONG_SWITCH", 5 : "TELEPORT"}

In [9]:
class Position:
  def __init__(self, x, y):
    self.x = x
    self.y = y

  # Overide the comparator
  def __eq__(self, other):
    return self.x == other.x and self.y == other.y
  def __lt__(self, other):
    return self.x**2 + self.y**2 < other.x**2 + other.y**2

In [ ]:
# Position of the block in the map
class Block:
  def __init__(self, parts, parent = None, move_from_parent = None):
    self.parts = sorted(parts)
    self.parent = parent
    self.move_from_parent = move_from_parent
    self.state = self.get_state()

  def get_state(self):
    row_diff = abs(self.parts[0][0] - self.parts[1][0])
    col_diff = abs(self.parts[0][1] - self.parts[1][1])

    if row_diff == 0 and col_diff == 0: return 0
    elif row_diff == 0 and col_diff == 1: return 1
    elif row_diff == 1 and col_diff == 0: return 2
    else: return 3
  
  def move(self, instruction, movable_part = None):
    if instruction == 0: return copy.copy(self)
    else: 
      affected_axis = abs(instruction)-1
      parts = copy.deepcopy(self.parts)
      product = abs(instruction*self.state)

      if self.state == 3:
        parts[movable_part][affected_axis] += np.sign(instruction)
      else:
        if product == 0:
        # Standing
          parts[0][affected_axis] += np.sign(instruction)*2
          parts[1][affected_axis] += np.sign(instruction)
        elif product == 2:
        # Apply max/min
          if instruction < 0: 
            parts[0][affected_axis] = min(parts[0][affected_axis], parts[1][affected_axis]) - 1
            parts[1][affected_axis] = parts[0][affected_axis]
          else:
            parts[0][affected_axis] = max(parts[0][affected_axis], parts[1][affected_axis]) + 1
            parts[1][affected_axis] = parts[0][affected_axis]
        else:
          parts[0][affected_axis] += np.sign(instruction)
          parts[1][affected_axis] += np.sign(instruction)
        
      if self.state == 3: return Block(parts, self, (instruction, movable_part))
      else: return Block(parts, self, instruction)

In [ ]:
class Special_button:
  def __init__ (self, name, position, effect_positions):
    self.name = name
    self.postion = position
    self.affects = affects

In [ ]:
class State:
  def __init__(self, block, special_button_state):
    self.block = block
    self.special_button_state = special_button_state

  # two state is considered equal if their parts and special_button_state array are equal
  def __eq__(self, other):
    return self.block.parts == other.block.parts and self.special_button_state == other.special_button_state

In [ ]:
class BLOXORZ:
  def __init__(self, map, initial_block, special_buttons, current_state = None):
    self.map = map
    self.special_buttons = special_buttons
    self.initial_state = State(block, [0]*len(special_buttons))
  
    self.goal = [(index, row.index(10)) for index, row in enumerate(map) if 10 in row][0]
    self.current =self.initial_state

  def update_state(self, current = None):
    if current == None: current = self.current

  def expand(self, current = None):
    if current == None: current = self.current
    children = []
    for instruction in [-1,1,-2,2]:
      child = current.move(instruction)
      if self.is_valid(child): children.append(child)
    return children


In [ ]:
 # This is the original DFS
def helper(game, stack, visited, loop):
  while len(stack) > 0:
    game.current = stack.pop()
    visited.append(game.current.parts)
    children = game.expand() 
    # if game.current.parent != None and game.current.parent.parts == [[0, 9], [1, 9]]: 
    # if game.current.parent != None:
    #   print(game.current.parent.parts, move[game.current.move_from_parent], game.current.parts)
    for child in children:
      
      if game.is_goal(child): 
        print("Goal:", child.parts)
        return child, loop
      if child.parts not in visited: stack.append(child)

    loop = loop + 1
  return -1, loop

#This is used to call DFS more conveniently
def DFS(game):
  index = 0;
  goal, loop = helper(game, [game.initial_block], [], 0)
  if goal == -1: print("No solutions")
  else:
    path = [goal]
    while path[-1].parent != None: path.append(path[-1].parent)
    path.reverse()
    print("Goal reached after", loop, "loops")
    for position in path: 
      print("Steps", index, move[position.move_from_parent],":", position.parts)
      index = index + 1

In [ ]:
level1 = BLOXORZ([[2,2,2,0,0,0,0,0,0,0],
                  [2,2,2,2,2,2,0,0,0,0],
                  [2,2,2,2,2,2,2,2,2,0],
                  [0,2,2,2,2,2,2,2,2,2], 
                  [0,0,0,0,0,2,2,10,2,2], 
                  [0,0,0,0,0,0,2,2,2,0]], Block([[1,1],[1,1]]))

level2 = BLOXORZ([[0,0,0,0,0,0,2,2,2,2,0,0,2,2,2],
                  [2,2,2,2,0,0,2,2,2,2,0,0,2,10,2],
                  [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                  [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                  [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
                  [2,2,2,2,0,0,2,2,2,2,0,0,0,0,0]], Block([[3,1],[3,1]]))

level3 = BLOXORZ([[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
                  [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
                  [2,2,2,2,2,2,2,2,2,0,0,2,2,10,2],
                  [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
                  [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]], Block([[2,1],[2,1]]))

level4 = BLOXORZ([[0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                  [0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                  [2,2,2,2,0,0,0,0,0,2,2,2,0,0],
                  [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                  [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                  [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                  [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                  [0,0,0,0,0,2,10,2,0,0,1,1,2,1],
                  [0,0,0,0,0,2,2,2,0,0,1,1,1,1]], Block([[5,1],[5,1]]))

level5 = BLOXORZ([[0,0,0,0,0,0,0,0,0,0,0,2,2,2,2],
                  [0,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
                  [0,2,2,2,2,0,0,0,0,0,0,0,2,2,2],
                  [0,2,2,2,2,0,0,0,0,0,0,0,0,0,0],
                  [0,2,2,2,2,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,2,2,2,2,2,2,2,2,2,2,0,0],
                  [0,0,0,0,0,0,0,0,0,0,2,2,2,2,2],
                  [2,2,2,0,0,0,0,0,0,0,2,2,2,2,2],
                  [2,10,2,2,2,2,2,2,2,2,2,2,2,0,0],
                  [2,2,2,2,2,0,0,0,0,0,0,0,0,0,0]], Block([[1,12],[1,12]]))

level6 = BLOXORZ([[0,0,0,0,0,2,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,2,0,0,2,2,2,0,0,0,0],
                  [0,0,0,0,0,2,0,0,2,2,2,2,2,0,0],
                  [2,2,2,2,2,2,0,0,0,0,0,2,2,2,2],
                  [0,0,0,0,2,2,2,0,0,0,0,2,2,10,2],
                  [0,0,0,0,2,2,2,0,0,0,0,0,2,2,2],
                  [0,0,0,0,0,0,2,0,0,2,2,0,0,0,0],
                  [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,0,0,2,2,2,0,0,0,0,0]], Block([[3,0],[3,0]]))


In [ ]:
DFS(level6)

In [ ]:
print(level4.map[0][2])
print(level4.is_valid(Block([[0,2],[1,2]])))

0
False


In [ ]:
block = Block([[5,1],[5,1]])
print(block.move(-1).parts)

[[3, 1], [4, 1]]


In [ ]:
level4.is_valid(Block([[3,1],[4,1]]))

True

In [ ]:
print(level2.initial_block.parts)

[[3, 1], [3, 1]]


In [ ]:
class Genetic():
  population = []
  prob = []
  def __init__(self, game, chromosome_length, population_capacity):
    self.game = game
    self.chromosome_length = chromosome_length
    self.population_capacity = population_capacity
    self.initPopulation()
    self.probCal()

  def initIndividual(self):
    chromosome = []
    for i in range(self.chromosome_length):
      # if chromosome is empty or having NONE as the last gen, add an arbitrary
      if len(chromosome) == 0 or chromosome[-1] == 0: chromosome.append(random.choice((0,1,2,3,4)))
      # if the last gen is UP, avoid DOWN
      elif chromosome[-1] == 1: chromosome.append(random.choice((0,1,3,4)))
      # if the last gen is DOWN, avoid UP
      elif chromosome[-1] == 2: chromosome.append(random.choice((0,2,3,4)))
      # if the last gen is LEFT, avoid RIGHT
      elif chromosome[-1] == 3: chromosome.append(random.choice((0,1,2,3)))
      # if the last gen is LEFT, avoid RIGHT
      else: chromosome.append(random.choice((0,1,2,4)))
    return chromosome
  
  def initPopulation(self):
    while len(self.population) < self.population_capacity:
      individual = self.initIndividual()
      final = self.game.move_process(individual)
      if final != -1: self.population.append(individual)

  def probCal(self):
    for individual in self.population:
      final = self.game.move_process(individual)
      self.prob.append(self.game.cost_to_goal(final))
    _sum = sum(self.prob)
    for i in range(len(self.prob)): self.prob[i] = self.prob[i] / _sum;

  #Seclect a parent in population
  def select(self):
    rannum = random.uniform(0, 1)
    for i in range (self.population_capacity):
      rannum = rannum - self.prob[i]
      if rannum < 0: return self.population[i-1]

  def cross_over(self):
    parent1 = self.select()
    parent2 = self.select()

    ran_index = random.ranint(0, self.chromosome_length)

    for i in range(ran_index, self.chromosome_length): parent1[i] = parent2[i]

    return parent1

  def mutate(self, chromosome):
     ran_index = random.ranint(0, self.chromosome_length)
     chromosome[ran_index] = random.choice((0,1,2,3,4))

  def populationUpdate(self):
    next_population = []

    # Cross over
    while (len(next_population) < self.population_capacity):
      child = self.cross_over()
      if self.game.move_process(child) != -1: next_population.append(child)
    # It is better no mutation
    
    for child in next_population: self.mutate(child)

    self.population = next_population
    self.probCal

  # Loop for 100 times
  def go(self):
    for i in range(100):
      for individual in self.population:
        if self.game.move_process(individual).is_goal(): return individual
      self.populationUpdate()

    return "Cannot find Solution"

In [15]:
a = [1,2,3,4,5]
b = [1,2,3,4,5]

print(a == b)

True
